# Testing no_std static lib is difficult...

In [2]:
cd ..

In [5]:
git log -1 -p

commit 06845495457b5d29a5827d629f0e61820ae0a73f (HEAD -> tsurai_test)
Author: Takaya Saeki <abc.tkys+pub@gmail.com>
Date:   Sun Jun 30 17:56:58 2019 +0900

    test_runner

diff --git a/src/lib.rs b/src/lib.rs
index cb90d46..a005f16 100755
--- a/src/lib.rs
+++ b/src/lib.rs
@@ -1,6 +1,10 @@
 #![no_std]
 #![feature(abi_x86_interrupt)]
 #![feature(asm)]
+#![feature(lang_items)]
+#![feature(custom_test_frameworks)]
+#![test_runner(crate::test_runner)]
+#![reexport_test_harness_main = "test_main"]
 
 use core::panic::PanicInfo;
 
@@ -8,7 +12,6 @@ extern crate multiboot2;
 
 pub mod interrupts;
 pub mod gdt;
-
 mod vga_buffer;
 
 #[no_mangle]
@@ -18,6 +21,9 @@ pub extern "C" fn rust_start(multiboot_info_addr: usize) -> ! {
     gdt::init();
     interrupts::init_idt();
 
+    #[cfg(test)]
+    test_main();
+
     println!("multiboot_info_addr: 0x{:x}", multiboot_info_addr);
     let boot_info = unsafe { multiboot2::load(multiboot_info_addr + 0xffffffffc0000000usize) };
     let memory_map_ta

Added `test_runner` 

In [4]:
cargo xtest --target=x86_64-rusty_l4.json 

   Compiling rusty_l4 v0.1.0 (/mnt/c/Users/abctk/Dropbox/develop/rust/l4/rusty_l4)
  --> src/vga_buffer.rs:71:21
   |
71 |                 0x20...0x7e | b'\n' => self.write_byte(byte),
   |                     ^^^ help: use `..=` for an inclusive range
   |
   = note: #[warn(ellipsis_inclusive_range_patterns)] on by default

  --> src/interrupts.rs:38:1
   |
38 | / extern "x86-interrupt" fn fault_handler(
39 | |     stack_frame: &mut InterruptStackFrame,
40 | |     _: PageFaultErrorCode) 
41 | | {
42 | |     println!("EXCEPTION: \n{:#?}", stack_frame);
43 | | }
   | |_^
   |
   = note: #[warn(dead_code)] on by default

error: requires `start` lang_item===================>   ] 19/20

error: aborting due to previous error===============>   ] 19/20

error: Could not compile `rusty_l4`.============================>   ] 19/20

To learn more, run the command again with --verbose.


: 101

However, build fails with the error `requires start lang_item`

In [6]:
git diff src

diff --git a/src/lib.rs b/src/lib.rs
index a005f16..e46b5f5 100755
--- a/src/lib.rs
+++ b/src/lib.rs
@@ -1,6 +1,7 @@
 #![no_std]
 #![feature(abi_x86_interrupt)]
 #![feature(asm)]
+#![feature(start)]
 #![feature(lang_items)]
 #![feature(custom_test_frameworks)]
 #![test_runner(crate::test_runner)]
@@ -15,6 +16,7 @@ pub mod gdt;
 mod vga_buffer;
 
 #[no_mangle]
+#[start]
 pub extern "C" fn rust_start(multiboot_info_addr: usize) -> ! {
     println!("Hello Hello, World!\nsome numbers: {} {}", 42, 1.337);
 


In [7]:
cargo xtest --target=x86_64-rusty_l4.json 

   Compiling rusty_l4 v0.1.0 (/mnt/c/Users/abctk/Dropbox/develop/rust/l4/rusty_l4)
  --> src/vga_buffer.rs:71:21
   |
71 |                 0x20...0x7e | b'\n' => self.write_byte(byte),
   |                     ^^^ help: use `..=` for an inclusive range
   |
   = note: #[warn(ellipsis_inclusive_range_patterns)] on by default

  --> src/interrupts.rs:38:1
   |
38 | / extern "x86-interrupt" fn fault_handler(
39 | |     stack_frame: &mut InterruptStackFrame,
40 | |     _: PageFaultErrorCode) 
41 | | {
42 | |     println!("EXCEPTION: \n{:#?}", stack_frame);
43 | | }
   | |_^
   |
   = note: #[warn(dead_code)] on by default

error: requires `start` lang_item===================>   ] 19/20

error: aborting due to previous error

error: Could not compile `rusty_l4`.============================>   ] 19/20

To learn more, run the command again with --verbose.


: 101

Added `start` (I don't know if this is meaningful at all), but no help.

In [8]:
git diff

diff --git a/Makefile b/Makefile
index 2de8a09..274c460 100755
--- a/Makefile
+++ b/Makefile
@@ -1,7 +1,9 @@
 all: run
 
-build/kernel: build/startup | build/
+cargo:
 	cargo xbuild --target x86_64-rusty_l4.json
+
+build/kernel: build/startup cargo | build/
 	cp target/x86_64-rusty_l4/debug/librusty_l4.a build/
 	ld.lld -Tsrc/linker.ld -o build/kernel-nonstripped build/startup build/librusty_l4.a
 	objcopy -g build/kernel-nonstripped build/kernel
@@ -33,4 +35,4 @@ clean:
 	rm -rf build
 	cargo clean
 
-.PHONY: run all clean build/kernel build
+.PHONY: run all clean build/kernel build cargo
diff --git a/src/lib.rs b/src/lib.rs
index a005f16..298c021 100755
--- a/src/lib.rs
+++ b/src/lib.rs
@@ -1,10 +1,12 @@
 #![no_std]
 #![feature(abi_x86_interrupt)]
 #![feature(asm)]
+#![feature(start)]
 #![feature(lang_items)]
 #![feature(custom_test_frameworks)]
 #![test_runner(crate::test_runner)]
 #![reexport_test_harness_main = "test_main"]
+#![no_main]
 
 use core::panic::PanicInfo;
 
@@ -15,6 +1

In [9]:
cargo xtest --target=x86_64-rusty_l4.json 

    Finished dev [unoptimized + debuginfo] target(s) in 0.05s
     Running target/x86_64-rusty_l4/debug/deps/rusty_l4-e1191528667ca90d
test /mnt/c/Users/abctk/Dropbox/develop/rust/l4/rusty_l4/target/x86_64-rusty_l4/debug/deps/rusty_l4-e1191528667ca90d


Adding `no_main` makes build succeed, However

In [10]:
readelf -S /mnt/c/Users/abctk/Dropbox/develop/rust/l4/rusty_l4/target/x86_64-rusty_l4/debug/deps/rusty_l4-e1191528667ca90d

There are 15 section headers, starting at offset 0x405b60:

Section Headers:
  [Nr] Name              Type             Address           Offset
       Size              EntSize          Flags  Link  Info  Align
  [ 0]                   NULL             0000000000000000  00000000
       0000000000000000  0000000000000000           0     0     0
  [ 1] .debug_str        PROGBITS         0000000000000000  000000e8
       000000000015db6c  0000000000000001  MS       0     0     1
  [ 2] .debug_abbrev     PROGBITS         0000000000000000  0015dc54
       0000000000006260  0000000000000000           0     0     1
  [ 3] .debug_info       PROGBITS         0000000000000000  00163eb4
       00000000000f3d2f  0000000000000000           0     0     1
  [ 4] .debug_ranges     PROGBITS         0000000000000000  00257be3
       000000000003e860  0000000000000000           0     0     1
  [ 5] .debug_macinfo    PROGBITS         0000000000000000  00296443
       000000000000005e  0000000000000000    

No `.text` section is emitted. All functions disappeared.This is understandable because there is no entry point due to `no_main`...